In [1]:
#importing libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
from tqdm import tqdm
from fuzzywuzzy import fuzz

E:\Users\ashti\anaconda3\envs\py3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def Punctuation(string): 
  
    # punctuation marks 
    punctuations = '''.'''
  
    # traverse the given string and if any punctuation 
    # marks occur replace it with null 
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 
  
    return string

In [3]:
#getting links from advocatekhoj for the act content
def get_adv_urls():
    '''Getting the URLs from advocatekhoj. These URLs are stored as a tuple (name, link)
    which are later mapped to the links in the India Code list. These URLs will be iterated
    on to fetch the content from the website.
    '''
    
    urllist = []
    FETCH_URL="https://www.advocatekhoj.com/library/bareacts/index.php"
    ROOT_URL=FETCH_URL.split("index.php")[0]
    
    rr=requests.get(FETCH_URL)
#     print(rr)
    rr_soup=BeautifulSoup(rr.text,'html.parser')
    
    table = rr_soup.find('table', cellpadding = "3", cellspacing= "0", align = "left", width = "100%")
    links = table.find_all('tr')
    for l in links:
        tempurl = l.a.attrs.get('href')
        name = l.a.font.get_text().strip()
        res = re.sub(r'[^\w\s]', '', name)
        res = res.strip()
        urllist.append((ROOT_URL+tempurl,res))
    
    return urllist

In [4]:
#getting links from India Code for act details
def get_indiacode_urls():
    '''
    Getting the URLs from the India Code website. These URLs are also stored as a tuple (name, link). 
    The act details are scraped from this website since they are not available in advocate khoj.
    '''
    
    url = 'https://indiacode.nic.in/handle/123456789/1362/browse?type=shorttitle&sort_by=1&order=DESC&rpp=100&etal=-1&null=&offset='
    baseurl = 'https://indiacode.nic.in'
    urllist = []
    namelist = []
    i = 0
    while(i<900):
        url += str(i).zfill(3)
        try:
            r = requests.get(url)
        except(requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, requests.exceptions.InvalidURL,requests.exceptions.InvalidSchema):
            print("Request Error")
            return
        soup = BeautifulSoup(r.content, 'html.parser')
        # print(soup.prettify())
        links = soup.find_all('div', class_ = 'panel panel-primary table-responsive')[0].find_all('a')
        name = soup.find_all('strong')
        name=[j.string for j in name]
        namelist.append(name)
#         print(name)
#         print(name)
        
        for l in links:
#             n = n.strip()
#             res = re.sub(r'[^\w\s]', '', n)
#             res = res.strip()
            tempurl = l.get('href')
#             print("\n")
#             print([n,baseurl+tempurl,len(tempurl)])
            if len(tempurl) < 90:
#                 print("\n")
#                 print([n,baseurl+tempurl,len(tempurl)])
                urllist.append(baseurl+tempurl)
        print(len(urllist))
        i += 100
        url = url[:-3]
    return urllist,namelist

In [42]:
adv_links = get_adv_urls()
adv_links

[('https://www.advocatekhoj.com/library/bareacts/abductedpersons/index.php?Title=Abducted Persons (Recovery and Restoration) Continuance Act, 1955\r',
  'Abducted Persons Recovery and Restoration Continuance Act 1955'),
 ('https://www.advocatekhoj.com/library/bareacts/absorbedareas/index.php?Title=Absorbed Areas (Laws) Act, 1954\r',
  'Absorbed Areas Laws Act 1954'),
 ('https://www.advocatekhoj.com/library/bareacts/acquiredterritories/index.php?Title=Acquired Territories (Merger) Act, 1960\r',
  'Acquired Territories Merger Act 1960'),
 ('https://www.advocatekhoj.com/library/bareacts/actuaries/index.php?Title=Actuaries Act, 2006\r',
  'Actuaries Act 2006'),
 ('https://www.advocatekhoj.com/library/bareacts/additionaldutiesofexcise/index.php?Title=Additional Duties of Excise (Textiles and Textile Articles) Act, 1978\r',
  'Additional Duties of Excise Textiles and Textile Articles Act 1978'),
 ('https://www.advocatekhoj.com/library/bareacts/admi/index.php?Title=Administrator - Generals Ac

In [34]:
ind_code_links, ind_code_name = get_indiacode_urls()

100
200
300
400
500
600
700
800
857


In [35]:
year_set = {2000,2001}
for i in range(1000,2100):
    year_set.add(i)
1973 in year_set

True

In [57]:
ind_lst = []
ind_code_links_final = ind_code_links.copy()
for i in ind_code_name:
    for j in range(0,len(i)):
        if i[j] != "Short Title":
            ind_lst.append(i[j])
for idx in range(0,len(ind_lst)):
    ind_code_links_final[idx] = (ind_code_links[idx],ind_lst[idx])
ind_code_links_final

[('https://indiacode.nic.in/handle/123456789/15619?view_type=browse&sam_handle=123456789/1362',
  'The Rashtriya Raksha University Act, 2020.'),
 ('https://indiacode.nic.in/handle/123456789/15623?view_type=browse&sam_handle=123456789/1362',
  'The National Forensic Sciences University Act, 2020'),
 ('https://indiacode.nic.in/handle/123456789/15618?view_type=browse&sam_handle=123456789/1362',
  'The Bilateral Netting of Qualified Financial Contracts Act, 2020'),
 ('https://indiacode.nic.in/handle/123456789/15512?view_type=browse&sam_handle=123456789/1362',
  'The Jammu and Kashmir Official Languages Act, 2020.'),
 ('https://indiacode.nic.in/handle/123456789/15511?view_type=browse&sam_handle=123456789/1362',
  'The Farmers (Empowerment and Protection) Agreement on Price Assurance and Farm Services Act, 2020.'),
 ('https://indiacode.nic.in/handle/123456789/15513?view_type=browse&sam_handle=123456789/1362',
  'The Farmers  Produce Trade and commerce (Promotion and Facilitation) Act, 2020')

In [60]:
def createdict(year_set, lst):
    dictionary = {}
    for year in year_set:
        act_lst = []
        for l,n in lst:
            if str(year) in n:
                act_lst.append([l,n])
        dictionary[year] = act_lst
    return dictionary

adv_dict = createdict(year_set, adv_links)
ind_dict = createdict(year_set, ind_code_links_final)

ind_dict

{2048: [],
 2049: [],
 2050: [],
 2051: [],
 2052: [],
 2053: [],
 2054: [],
 2055: [],
 2056: [],
 2057: [],
 2058: [],
 2059: [],
 2060: [],
 2061: [],
 2062: [],
 2063: [],
 2064: [],
 2065: [],
 2066: [],
 2067: [],
 2068: [],
 2069: [],
 2070: [],
 2071: [],
 2072: [],
 2073: [],
 2074: [],
 2075: [],
 2076: [],
 2077: [],
 2078: [],
 2079: [],
 2080: [],
 2081: [],
 2082: [],
 2083: [],
 2084: [],
 2085: [],
 2086: [],
 2087: [],
 2088: [],
 2089: [],
 2090: [],
 2091: [],
 2092: [],
 2093: [],
 2094: [],
 2095: [],
 2096: [],
 2097: [],
 2098: [],
 2099: [],
 1000: [],
 1001: [],
 1002: [],
 1003: [],
 1004: [],
 1005: [],
 1006: [],
 1007: [],
 1008: [],
 1009: [],
 1010: [],
 1011: [],
 1012: [],
 1013: [],
 1014: [],
 1015: [],
 1016: [],
 1017: [],
 1018: [],
 1019: [],
 1020: [],
 1021: [],
 1022: [],
 1023: [],
 1024: [],
 1025: [],
 1026: [],
 1027: [],
 1028: [],
 1029: [],
 1030: [],
 1031: [],
 1032: [],
 1033: [],
 1034: [],
 1035: [],
 1036: [],
 1037: [],
 1038: [],

In [61]:
final_link = []
for a_keys in adv_dict:
#     print(adv_dict[a_keys])
    for l,n in adv_dict[a_keys]:
        for i_l, i_n in ind_dict[a_keys]:
            ratio = fuzz.ratio(n.lower(),i_n.lower())
            if ratio > 85:
                final_link.append([i_n, i_l, l])

In [71]:
final_link[49]

['The Guardians and Wards Act, 1890',
 'https://indiacode.nic.in/handle/123456789/2318?view_type=browse&sam_handle=123456789/1362',
 'https://www.advocatekhoj.com/library/bareacts/guardiansandwards/index.php?Title=Guardians and Wards Act, 1890\r']

In [72]:
#Scraping act content from advocatekhoj and india code to give final json
def adv_scraper(name, ind_link, adv_link):
    
    #India Code
    mainlist = []
    act = {}
    chapter = []
    year = ''
    actno = ''
    name = ''
    enact = ''
    enforce = ''
    ministry = ''
    dep = ''
    summary = ''
    try:
        r = requests.get(ind_link)
    except(requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, requests.exceptions.InvalidURL,requests.exceptions.InvalidSchema):
        print("Request Error")
        time.sleep(20)
        scrapesite(adv_link)
        return
    soup = BeautifulSoup(r.content, 'html.parser')
    details = soup.find_all('table', class_ = 'table itemDisplayTable')[0].find_all('tr')
    info = []
    for d in details:
        info.append( d.text.split(u':'))
    # print(info)
    for i in info:
        if i[0] == 'Act Year':
            year = i[1].replace(u'\xa0', u'')
        if i[0] == 'Act Number':
            actno = i[1].replace(u'\xa0', u'')
        if i[0] == 'Short Title':
            name = i[1].replace(u'\xa0', u'')
        if i[0] == 'Enactment Date':
            enact = i[1].replace(u'\xa0', u'')
        if i[0] == 'Enforcement Date':
            enforce = i[1].replace(u'\xa0', u'')
        if i[0] == 'Ministry':
            ministry = i[1].replace(u'\xa0', u'')
        if i[0] == 'Department':
            dep = i[1].replace(u'\xa0', u'')
        if i[0] == 'Long Title':
            summary = i[1].replace(u'\xa0', u'')
#     print(year, actno, name, enact, enforce, ministry,dep,summary)
    name = Punctuation(str(name))
    final_act = {}
    final_act['Year'] = str(year)
    final_act['Act No'] = str(actno)
    final_act['Act Name'] = str(name)
    final_act['Enactment Date'] = str(enact)
    final_act['Enforcement Date'] = str(enforce)
    final_act['Ministry'] = str(ministry)
    final_act['Department'] = str(dep)
    final_act['Summary'] = str(summary)
    
    # advocatekhoj
    act_url = adv_link
    root_url=act_url.split("index.php")[0]
    rr=requests.get(act_url)
    rr_soup=BeautifulSoup(rr.text,'html.parser')
    html_code=rr_soup.find("table",cellspacing="2",cellpadding="2",width="100%")
    soup=html_code
    rows=soup.find_all("tr")
    y = []
    chap = None
    x = {}
    counter = 0
    no_chap_flag = False
    for i in rows[2:]:
        old_chap = chap
        if i.td.attrs.get("colspan") is not None:
            y.pop(0)
            y.append(x)
            break
        if i.td.font is None:
            continue
        if i.td.font.b is not None:
            trows = i.find_all("td")
            chap = trows[0].string if trows[0].string is not None else "None"
            chn = trows[1].string
            y.append(x)
            x = {}
            x["chapter_no"] = chap
            x["chapter_name"] = chn
            x["all_sections"] = {}
            no_chap_flag = True
        else:
            if no_chap_flag != True:
                chap = "None"
                chn = "None"
                y.append(x)
                x = {}
                x["chapter_no"] = chap
                x["chapter_name"] = chn
                x["all_sections"] = {}
                old_chap = chap
                no_chap_flag = True
                
        if old_chap == chap:
            if i.td.find_next_sibling().a is None:
                x["all_sections"][f"{i.td.string}"] = {
                    "heading": i.td.find_next_sibling().get_text(),
                    "content": "0",
                }
            else:
                #            (ROOT_URL+i.td.find_next_sibling().a.attrs["href"])
                resp = requests.get(root_url + i.td.find_next_sibling().a.attrs["href"])
                resp_soup = BeautifulSoup(resp.text, "html.parser")
                d = resp_soup.find_all("p", align="justify")
                d.pop(0)
                f = [a.get_text().strip() for a in d]
                key = i.td.string if i.td.string is not None else "None"
                x["all_sections"][key] = {
                    "heading": i.td.find_next_sibling().get_text().strip(),
                    "content": " ".join(f),
                }
        counter += 1
        if counter == len(rows[2:]) - 1:
            y.pop(0)
            y.append(x)
    final_act["Chapters"] = y
    with open(f"{str(year)}-{str(actno)}-{str(name)}.json", 'w') as f:
        string = json.dumps(final_act)
        f.write(string)
    return final_act

In [74]:
# loop for iterating through the list of links and getting the jsons for all acts
for i in range(0,len(final_link)):
    try:
        print(i)
        print("\n")
        final_act = adv_scraper(final_link[i][0],final_link[i][1],final_link[i][2])
        print("\n")
        print(final_link[i][0],final_link[i][1],final_link[i][2])
    except:
        print("\n")
        print("Check Here")
        pass

0


1850 18 The Judicial Officers Protection Act, 1850 1850-04-04 04-04 1850 Ministry of Law and Justice Department of Justice An Act for the protection of Judicial Officers.


The Judicial Officers Protection Act, 1850 https://indiacode.nic.in/handle/123456789/2273?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/judicialofficersprotection/index.php?Title=Judicial Officers Protection Act, 1850
1


1851 08 The Indian Tolls Act, 1851 1851-06-04 04-06-1851 Ministry of Road Transport and Highways  An Act for enabling Government to levy Tolls on Public Roads and Bridges.


The Indian Tolls Act, 1851 https://indiacode.nic.in/handle/123456789/2274?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/indiantolls1851/index.php?Title=Indian Tolls Act, 1851
2


1855 13 The Fetal Accidents Act, 1855 1855-03-27 27-03-1855 Ministry of Law and Justice Legislative Department An Act to provide compensation to families for loss



The Indian Contract Act, 1872 https://indiacode.nic.in/handle/123456789/2187?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/indiancontract/index.php?Title=Indian Contract Act, 1872
20


1872 01 The Indian Evidence Act, 1872 1872-03-15 01-09-1872 Ministry of Law and Justice Legislative Department To consolidate, define and amend the law of Evidence.


The Indian Evidence Act, 1872 https://indiacode.nic.in/handle/123456789/2188?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/indianevidence/index.php?Title=Indian Evidence Act, 1872
21


1873 03 The Madras Civil Courts Act, 1873 1873-01-21 21-01-1873 Ministry of Law and Justice Department of Justice An Act to consolidate and amend the law realating to the Civil Courts of the Madras Presidency subordinate to the High Court.


The Madras Civil Courts Act, 1873 https://indiacode.nic.in/handle/123456789/2299?view_type=browse&sam_handle=123456789/1362 https://w

1883 20 The Punjab District Boards Act, 1883 1883-10-12 12-10-1883 Ministry of Law and Justice Legislative Department An Act to make better provision for local selfgovernment in the Districts of the Panjab.


The Punjab District Boards Act, 1883 https://indiacode.nic.in/handle/123456789/2393?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/panjabdistrictboards/index.php?Title=Panjab District Boards Act, 1883
39


1884 04 The Explosives Act, 1884 1884-02-26 01-07-1887 Ministry of Commerce and Industry Department of Promotion of Industry and Internal Trade An Act to regulate the manufacture, possession, use, sale,transport, import and export of explosives.


The Explosives Act, 1884 https://indiacode.nic.in/handle/123456789/2301?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/explosives/index.php?Title=Explosives Act, 1884
40


1885 13 The Indian Telegraph Act, 1885 1885-07-22 01-10-1885 Ministry of Communic

1897 03 The Epidemic Diseases Act, 1897 1897-02-04 04-02-1897 Ministry of Health and Family Welfare Department of Health and Family Welfare An Act to provide for the better prevention of the spread of Dangerous Epidemic Diseases.


The Epidemic Diseases Act, 1897 https://indiacode.nic.in/handle/123456789/2326?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/epidemicdiseases/index.php?Title=Epidemic Diseases Act, 1897
57


1897 10 The General Clauses Act, 1897 1897-03-11 11-03-1897 Ministry of Law and Justice Legislative Department An Act to consolidate and extend the General Clauses Act, 1868 and 1887.


Check Here
58


1898 06 The Indian Post Office Act, 1898 1898-03-22 01-07-1898 Ministry of Communications Department of Posts An Act to consolidate and amend the law relating to the Post Offices in India.


The Indian Post Office Act, 1898 https://indiacode.nic.in/handle/123456789/2329?view_type=browse&sam_handle=123456789/1362 https://www.advoca

1913 05 The White Phosphorus Matches Prohibition Act, 1913 1913-03-07 01-7-1913 (Except s. 6) 01-07-1914 Ministry of Home Affairs Department of States An Act to prohibit the importation, manufacture and sale of matches made with white phosphorus.


The White Phosphorus Matches Prohibition Act, 1913 https://indiacode.nic.in/handle/123456789/2353?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/whitephosphorusmatches/index.php?Title=White Phosphorus Matches Prohibition Act, 1913
76


1915 16 The Banaras Hindu University Act, 1915 1915-10-01 01-04-1916 Ministry of Human Resource Development Department of Higher Education An Act to establish and incorporate a teaching and residential Hindu University at Banaras.


The Banaras Hindu University Act, 1915 https://indiacode.nic.in/handle/123456789/2356?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/banarashinduuniversity/index.php?Title=Banaras Hindu University A

...



The Right to Fair Compensation and Transparency in Land Acquisition, Rehabilitation and Resettlement Act, 2013 https://indiacode.nic.in/handle/123456789/2121?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/righttofaircompensation/index.php?Title=Right to Fair Compensation and Transparency in Land Acquisition, Rehabilitation and Resettlement Act, 2013
687


2013 14 The Sexual Harassment of Women at Workplace (Prevention, Prohibition and Redressal) Act, 2013 2013-04-22 09-12-2013 Ministry of Women and Child Development  An Act to provide protection against sexual harassment of women at workplace and for the prevention and redressal of complaints of sexual harassment and for matters connected therewith or incidental thereto.


The Sexual Harassment of Women at Workplace (Prevention, Prohibition and Redressal) Act, 2013 https://indiacode.nic.in/handle/123456789/2104?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/ba

2016 38 The Compensatory Afforestation Fund Act, 2016. 2016-08-03  Ministry of Environment, Forest and Climate Change  An Act to provide for the establishment of funds under the public accounts of India and the  public accounts of each State and crediting thereto the monies received from the user agencies towards compensatory afforestation, additional compensatory afforestation, penal compensatory afforestation, net present value and all other amounts recovered from such agencies under the Forest (Conservation) Act, 1980, constitution of an authority at national level and at each of the State and Union territory Administration for administration of the funds and to utilise the monies so collected for undertaking artificial regeneration (plantations), assisted natural regeneration, protection of forests, forest related infrastructure development, Green India Programme, wildlife protection and other related activities and for matters connected therewith or incidental thereto.


The Compe



The Footwear Design and Development Institute Act, 2017 https://indiacode.nic.in/handle/123456789/2255?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/footweardesigndevelopment/index.php?Title=Footwear Design and Development Institute Act, 2017
710


2017 15 The Goods and Services Tax (Compensation to States) Act, 2017 2017-04-12 01-07-2017 Ministry of Finance Department of Revenue An Act to provide for compensation to the States for the loss of revenue arising on account of implementation of the goods and services tax in pursuance of the provisions of the Constitution (One Hundred and First Amendment) Act, 2016.


The Goods and Services Tax (Compensation to States) Act, 2017 https://indiacode.nic.in/handle/123456789/2253?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/goodsservicestax/index.php?Title=Goods and Services Tax (Compensation to States) Act, 2017
711


2017 16 The Human Immunodeficiency Viru

2019 50 The International Financial Services Centres Authority Act, 2019 2019-12-19 27th April, 2020 Sections 1 to 11 (both inclusive), 14, 15, 16, 27, 29, 30 and 34, vide notification No. S.O. 1384(E), dated 27th April, 2020, see Gazette of India, Extraordinary, Part II, sec. 3(ii). Ministry of Finance Department of Economic Affairs An Act to provide for the establishment of an Authority to develop and regulate the financial services market in the International Financial Services Centres in India and for matters connected therewith or incidental thereto.


The International Financial Services Centres Authority Act, 2019 https://indiacode.nic.in/handle/123456789/14009?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/internationalfinancial/index.php?Title=International Financial Services Centres Authority Act, 2019
723


2019 34 The Jammu and Kashmir Reorganisation Act, 2019 2019-08-09  Ministry of Home Affairs Department of Jammu and Kashmir Affa

2020 3 The Direct Tax Vivad se Vishwas Act, 2020 2020-03-17  Ministry of Finance Department of Revenue An Act to provide for resolution of disputed tax and for matters connected therewith or incidental thereto.


The Direct Tax Vivad se Vishwas Act, 2020 https://indiacode.nic.in/handle/123456789/13812?view_type=browse&sam_handle=123456789/1362 https://www.advocatekhoj.com/library/bareacts/directtaxvivad/index.php?Title=Direct Tax Vivad Se Vishwas Act, 2020
734


2020 20 The Farmers (Empowerment and Protection) Agreement on Price Assurance and Farm Services Act, 2020. 2020-09-24 05-06-2020 Ministry of Agriculture and Farmers Welfare Department of Agriculture, Cooperation and Farmers Welfare An Act to provide for a national framework on farming agreements that protects and empowers farmers to engage with agri-business firms, processors, wholesalers, exporters or large retailers for farm services and sale of future farming produce at a mutually agreed remunerative price framework in a fai

In [1]:
x =123

In [3]:
print(type(x))

<class 'int'>
